In [ ]:
import discord
from discord.ext import commands

import nest_asyncio
import os
nest_asyncio.apply()
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv('DISCORD_TOKEN')

intents = discord.Intents.all()
intents.message_content = True

client = discord.Client(intents=intents)

bot = commands.Bot(command_prefix='$', intents=intents)

def get_metadata_from_message(message):
    return {
        'message_id': message.id,
        'channel': message.channel,
        'author': message.author,
        'content': message.content,
    }
    
# @bot.command()
# async def test(ctx, arg1, arg2):
#     await ctx.send(f'You passed {arg1} and {arg2}')

@commands.command()
async def get_summary(ctx, *args):
    arguments = ', '.join(args)
    await ctx.send(f'{len(args)} arguments: {arguments}')    

@bot.event
async def on_ready():
    print(f'We have logged in as {bot.user}')

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if message.content.startswith('$hello'):
        print(message)
        print(get_metadata_from_message(message))
        await message.channel.send('Hello!')
    
bot.add_command(get_summary)
client.run(TOKEN)

[2024-11-18 13:06:19] [INFO    ] discord.client: logging in using static token
[2024-11-18 13:06:20] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 27ba5f647311f305da93f0ba3f8a3a75).


In [1]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4
from operator import itemgetter
import markdown
import datetime
import pytube
import os

async def web_loader(url: str):
    loader = WebBaseLoader(web_paths=[url])
    docs = []
    async for doc in loader.alazy_load():
        docs.append(doc)
    return docs

def chain_summarization(content: str, size: str = 'medium'):
    
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        |prompt
        | llm
        |StrOutputParser()
    )

    resume = chain.invoke({'texto': content})
    return resume

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# "\n".join([x.page_content for x in content])

In [4]:
content = await web_loader('https://www.uol.com.br')
trim_content = "\n".join([x.page_content for x in content])[:2_000]
summary = chain_summarization(trim_content, 'medium')
summary

'O conteúdo analisado refere-se ao portal UOL, que se apresenta como um "universo online" oferecendo uma variedade de serviços e informações. Aqui estão os principais pontos abordados:\n\n1. **Serviços Oferecidos**: O UOL disponibiliza diversos produtos e serviços, como UOL Play, PagBank, UOL Host e UOL Ads, que abrangem entretenimento, finanças, hospedagem e publicidade online. Isso permite que os usuários tenham acesso a uma gama de soluções em um único lugar.\n\n2. **Conteúdo Exclusivo**: O portal oferece uma assinatura que garante acesso ilimitado a conteúdo exclusivo, permitindo que os assinantes desfrutem de informações e serviços diferenciados.\n\n3. **Atendimento ao Cliente**: O UOL também disponibiliza canais de atendimento, como um número de telefone para contato, facilitando a comunicação com os usuários para suporte e esclarecimentos.\n\n4. **Variedade de Temas**: O site é dividido em várias seções, cobrindo temas como notícias, entretenimento, economia, saúde, esportes, e 

In [ ]:
import discord
from discord.ext import commands

import nest_asyncio
import os
nest_asyncio.apply()
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv('DISCORD_TOKEN')

intents = discord.Intents.all()
intents.message_content = True

bot = commands.Bot(command_prefix='$', intents=intents)

def get_metadata_from_message(message):
    return {
        'message_id': message.id,
        'channel': message.channel,
        'author': message.author,
        'content': message.content,
    }
    
@commands.command()
async def get_summary(ctx, *args):
    arguments = ', '.join(args)
    
    
    await ctx.send(f'Processando o link...')
    url = args[0]
    await ctx.send(f'Scrapping url: {args[0]}')
    try:
        content = await web_loader(url)
        trim_content = "\n".join([x.page_content for x in content])[:2_000]
        summary = chain_summarization(trim_content, 'medium')
        await ctx.send(summary)
    except:
        await ctx.send(f'Erro ao acessar a página {url}')
        return
    
bot.add_command(get_summary)
bot.run(TOKEN)

[2024-11-18 13:16:57] [INFO    ] discord.client: logging in using static token
[2024-11-18 13:16:58] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 3c78503aa97960dababd28e3dc468056).
